In [11]:
from scraper import number_companies, valuetoday_usa_scraper, df_cleaning

In [22]:
dataframe = valuetoday_usa_scraper(60)
final_df = df_cleaning(dataframe)

In [18]:
final_df

,Company Name,Company Business,Market Value,Annual Revenue,Operating Income,Net Income,Assets,Liabilities
0,APPLE,Technology,2256000000000,274510000000,66280000000,274510000000,317344000000,245062000000
1,MICROSOFT CORPORATION,Technology,1682000000000,143000000000,52950000000,143000000000,301311000000,183007000000
2,ALPHABET,Technology,1185000000000,182520000000,41220000000,182520000000,299243000000,86320000000
3,AMAZON.COM,eCommerce,1634000000000,386060000000,22890000000,386060000000,110908000000,184586000000
4,FACEBOOK,Communication Services,778040000000,85960000000,32670000000,85960000000,139691000000,29244000000
5,BERKSHIRE HATHAWAY,Financial Services,543679999999,327220000000,0,327220000000,829946000000,410730000000
6,J P MORGAN CHASE & CO,Financial Services,387330000000,122900000000,0,122900000000,3386000000000,3106000000000
7,VISA,Financial Services,482170000000,21840000000,14090000000,21840000000,77884000000,42360000000
8,JOHNSON & JOHNSON,Healthcare,414310000000,82580000000,19730000000,82580000000,158380000000,95402000000
9,WALMART,Retail,407840000000,523960000000,22540000000,523960000000,237382000000,162072000000


In [1]:
import requests
import re
import pandas as pd
import warnings
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [16]:
ua = UserAgent()
header = {'User-Agent':str(ua.chrome)}
url = "https://www.value.today/headquarters/united-states-america-usa?page=0"

source = requests.get(url, headers=header)
soup = BeautifulSoup(source.content, "html.parser")

# Create a list of divs with all required information.
info_blocks = soup.find_all("div", class_="node node--type-listed-companies node--view-mode-teaser ds-2col-stacked clearfix")

collected_names = []
collected_businesses = []
collected_values = []
collected_values_metrics = []
collected_revenues = []
collected_revenues_metrics = []
collected_op_incomes = []
collected_op_incomes_metrics = []
collected_net_incomes = []
collected_net_incomes_metrics = []
collected_assets = []
collected_assets_metrics = []
collected_liabilities = []
collected_liabilities_metrics = []

for item in info_blocks:

    def collect_values(item, collected_list):
        if len(item) == 0:
            collected_list.append(None)
        else:
            string = item[0].text.strip() 
            value = re.findall("\d+[.,]\d+|^\d+", string)
            collected_list.append(value[0])


    def collect_metrics(item, collected_list):
        if len(item) == 0:
            collected_list.append(None)
        else:
            string = item[0].text.strip()
            if "Billion" in string:
                collected_list.append("Billion")
            if "Million" in string:
                collected_list.append("Million")

    name = item.find("h2").text.strip()
    collected_names.append(name)

    business = item.select("div.field--name-field-company-category-primary > div > div.field--item > a")[0].text.strip()
    collected_businesses.append(business)

    market_value = item.select("div.field--name-field-market-value-jan012021 > div.field--item")[0]["content"]
    collected_values.append(market_value)

    market_value_item = item.select("div.field--name-field-market-value-jan012021 > div.field--item")
    collect_metrics(market_value_item, collected_values_metrics)

    revenue_item = item.select("div.field--name-field-annual-revenue > div.field--item")
    collect_values(revenue_item, collected_revenues)
    collect_metrics(revenue_item, collected_revenues_metrics)

    op_income_item = item.select("div.field--name-field-annual-operating-income > div.field--item")
    collect_values(op_income_item, collected_op_incomes)
    collect_metrics(op_income_item, collected_op_incomes_metrics)

    net_income_item = item.select("div.field--name-field-annual-revenue > div.field--item")
    collect_values(net_income_item, collected_net_incomes)
    collect_metrics(net_income_item, collected_net_incomes_metrics)

    assets_item = item.select("div.field--name-field-total-assets > div.field--item")
    collect_values(assets_item, collected_assets)
    collect_metrics(assets_item, collected_assets_metrics)

    liabilities_item = item.select("div.field--name-field-total-liabilities- > div.field--item")
    collect_values(liabilities_item, collected_liabilities)
    collect_metrics(liabilities_item, collected_liabilities_metrics)

    df = pd.DataFrame({"Company Name": collected_names, "Company Business": collected_businesses, "Market Value": collected_values, 
                        "Value Metric": collected_values_metrics, "Annual Revenue": collected_revenues, 
                        "Revenue Metric": collected_revenues_metrics, "Operating Income": collected_op_incomes, 
                        "Op Income Metric": collected_op_incomes_metrics, "Net Income": collected_net_incomes, 
                        "Net Income Metric": collected_net_incomes_metrics, "Assets": collected_assets, 
                        "Assets Metric": collected_assets_metrics, "Liabilities": collected_liabilities, 
                        "Liabilities Metric": collected_liabilities_metrics})

In [17]:
df

,Company Name,Company Business,Market Value,Value Metric,Annual Revenue,Revenue Metric,Operating Income,Op Income Metric,Net Income,Net Income Metric,Assets,Assets Metric,Liabilities,Liabilities Metric
0,APPLE,Technology,2256,Billion,274.51,Billion,66.28,Billion,274.51,Billion,"317,344",Million,"245,062",Million
1,MICROSOFT CORPORATION,Technology,1682,Billion,143.00,Billion,52.95,Billion,143.00,Billion,"301,311",Million,"183,007",Million
2,ALPHABET,Technology,1185,Billion,182.52,Billion,41.22,Billion,182.52,Billion,299.243,Billion,86.32,Billion
3,AMAZON.COM,eCommerce,1634,Billion,386.06,Billion,22.89,Billion,386.06,Billion,"110,908",Million,"184,586",Million
4,FACEBOOK,Communication Services,778.04,Billion,85.96,Billion,32.67,Billion,85.96,Billion,"139,691",Million,"29,244",Million
5,BERKSHIRE HATHAWAY,Financial Services,543.68,Billion,327.22,Billion,None,None,327.22,Billion,829.946,Billion,410.73,Billion
6,J P MORGAN CHASE & CO,Financial Services,387.33,Billion,122.90,Billion,None,None,122.90,Billion,"3,386",Billion,"3,106",Billion
7,VISA,Financial Services,482.17,Billion,21.84,Billion,14.09,Billion,21.84,Billion,"77,884",Million,"42,360",Million
8,JOHNSON & JOHNSON,Healthcare,414.31,Billion,82.58,Billion,19.73,Billion,82.58,Billion,"158,380",Million,"95,402",Million
9,WALMART,Retail,407.84,Billion,523.96,Billion,22.54,Billion,523.96,Billion,"237,382",Million,"162,072",Million


In [9]:
number = info_blocks[1].select("div.field--name-field-annual-operating-income > div.field--item")[0].text

In [10]:
number

'8.48 Billion USD for Year ending December-2020'

In [11]:
def collect_metrics(item, collected_list):
    if len(item) == 0:
        collected_list.append(None)
    else:
        string = item[0].text.strip()
    if "Billion" in string:
        collected_list.append("Billion")
    if "Million" in string:
        collected_list.append("Million")

op_income_item = item.select("div.field--name-field-annual-operating-income > div.field--item")
op_income_string = op_income_item[0].text.strip()



In [28]:
op_income_item = item.select("div.field--name-field-annual-operating-income > div.field--item")
if len(op_income_item) == 0:
    collected_op_incomes.append(None)
else:
    op_income_string = op_income_item[0].text.strip()
    op_income = re.findall("\d+[.,]\d+", op_income_string)
    collected_op_incomes.append(op_income[0])

if len(op_income_item) == 0:
    collected_op_incomes_metrics.append(None)
elif "Billion" in op_income_string:
    collected_op_incomes_metrics.append("Billion")
elif "Million" in op_income_string:
    collected_op_incomes_metrics.append("Million")

In [29]:
collected_op_incomes_metrics

['Billion',
 'Billion',
 'Billion',
 'Billion',
 'Billion',
 None,
 None,
 'Billion',
 'Billion',
 'Billion',
 'Billion']

In [31]:
collected_op_incomes

['66.28',
 '52.95',
 '41.22',
 '22.89',
 '32.67',
 None,
 None,
 '14.09',
 '19.73',
 '22.54',
 '22.54',
 '22.54']